In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import allantools

In [ ]:
pd.set_option("display.precision", 15)      # Show up to 15 decimal places
#baseName = 'fixedL1l52'
baseName = 'baseline1'
dirName  = 'labData/'
rowLimit = 2000 # Rows to keep after joining ticc and timTp samples

In [ ]:
# Read TAPR TICC data into a dataframe, as captured to a file by ticc.py running on a host.
# Events are rising edges of a PPS signal from a DUT, resulting in a timestamp on the TICC's reference clock.
# Columns:
#  ppsHostClock: Host clock when serial data for PPS event timestamp was read (ticc.py logs this in UTC)
#  ppsRefClock:  Reference clock when PPS event happened (elapsed seconds since TICC started)
#
# The frequency of the TICC reference clock comes from an external 10 MHz source, a Geppetto Electronics GNSSDO in my case.
# It should be almost exactly 1e7 times the PPS frequency, with an arbitray phase relationship.
# So we expect the whole number portion of ppsRefClock to increment by 1 every second, while the fractional seconds jitter around
# some slowly-changing phase offset.
# Therefore, there's very little information in the whole seconds, while the fractional seconds contain the most interesting data.
# And as the whole number grows with a floating point representation, precision is lost in the fractional digits.
# So once we confirm the whole number of seconds is behaving as expected, we can drop it and focus on the fractional seconds.
# Whatever slowly-changing phase offset exists, it won't impact the deviation metrics.
# We treat the fractional seconds as an instantaneous (but nosiy) measurement of the phase error against the ref clock.
#
# There is a very small, but non-zero chance that the static phase offset plus the jitter causes sequential PPS timestamps to be
# within the same second or more than one second apart, leading to missing or duplicate whole seconds.
# Instead of properly handling whole seconds when this happens, just fail on assertions.
def readTicc(chan):
    ticcFile = f"{dirName}/{baseName}.ticc{chan}.csv"
    ticcData = pd.read_csv(ticcFile, dtype={'ppsHostClock': str, 'ppsRefClock': str})

    # Convert host timestamp string to UTC timestamp
    ticcData["ppsHostClock"] = pd.to_datetime(ticcData.ppsHostClock, utc=True)

    # Split ppsRefClock string into whole and fractional seconds
    ticcData[["rcWhole", "rcFrac"]] = ticcData.ppsRefClock.str.split(".", n=1, expand=True)

    # Check for missing or duplicate whole seconds
    ticcData['rcWhole'] = ticcData['rcWhole'].astype(int)
    expected = set(range(ticcData.rcWhole.min(), ticcData.rcWhole.max() + 1))
    observed = set(ticcData.rcWhole)
    missing = sorted(expected - observed)
    duplicates = ticcData.rcWhole[ticcData.rcWhole.duplicated()].unique().tolist()
    assert len(missing)    == 0, f"Missing whole seconds in ticc{chan}: {missing}"
    assert len(duplicates) == 0, f"Duplicate whole seconds in ticc{chan}: {duplicates}"

    # Convert ref clock fractional part from digit string to float
    ticcData['rcFrac'] = "0." + ticcData['rcFrac'].astype(str)
    ticcData['rcFrac'] = ticcData['rcFrac'].astype(float)

    # Also get fractional part of host clock
    ticcData['hcFrac'] = (ticcData.ppsHostClock.astype('int64')-1e9*(ticcData.ppsHostClock.astype('int64')//1e9))/1e9

    # With overly careful consideration of maintining floating point precision, get interval between PPS events on ref clock.
    ticcData['rcTi'] = (ticcData.rcWhole-ticcData.rcWhole.shift(1)) + (ticcData.rcFrac - ticcData.rcFrac.shift(1)) # Time interval between refClock samples on ref clock

    ticcData['bn'] = baseName
    return ticcData

In [ ]:
ticcA = readTicc('')

In [ ]:
ticcA.dtypes

In [ ]:
ticcA

In [ ]:
#beg = 0
#end = 1000
beg = 10015
end = 10500
fMin = 1.0/(1.0-ticcA.rcTi[beg:end].min())
fMax = 1.0/(  ticcA.rcTi[beg:end].max()-1.0)
# XXX I suspect this is bogus
print(f"Frequency midpoint: {(fMax+fMin)/2e6} MHz")
print(f"Period midpoint: {1e12/((fMax+fMin)/2)} ps")

In [ ]:
from matplotlib.ticker import FuncFormatter
fig, ax = plt.subplots(figsize=(22, 6))
fig.canvas.draw()
bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
widthPx = int(bbox.width*fig.dpi)
print("Figure width in pixels:", widthPx)
print(fig.get_figwidth())
#plt.hist(ticcA[(ticcA.rcTi>0.999999999809074) & (ticcA.rcTi<0.999999999850000)][:99900].rcTi, bins=400, color='blue', alpha=0.7)
plt.hist(1e9*(ticcA.rcTi[beg:end]-1.0), bins=widthPx, color='blue', alpha=0.7)
plt.title('Histogram of Time Error Between PPS Pulses')
plt.xlabel('Time Error (ns)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))
plt.show()



In [ ]:
ticcA.rcTi[beg:end].describe()

In [ ]:
ticcA['tiMa10'] = ticcA.rcTi.rolling(10).mean()

In [ ]:
plt.figure(figsize=(22, 6))
plt.plot(ticcA.tiMa10[beg:end], 'o-', color='blue', alpha=0.7)

In [ ]:

# File format for .ticc.csv files logging PPS events
# Columns:
#  ppsHostClock: Host clock when PPS event timestamp arrvied (UTC)
#  ppsRefClock:  Reference clock when PPS event happened (zero-based count of elapsed seconds)

ticcA = pd.read_csv(f"{dirName}/{baseName}.ticcA.csv", dtype={'ppsHostClock': str, 'ppsRefClock': str})
ticcB = pd.read_csv(f"{dirName}/{baseName}.ticcB.csv")
ticcA['fn'] = baseName
ticcB['fn'] = baseName

# Convert string to datetime
ticcA["ppsHostClock"] = pd.to_datetime(ticcA.ppsHostClock, utc=True)
ticcB["ppsHostClock"] = pd.to_datetime(ticcB.ppsHostClock, utc=True)

# Assuming host clock sync is better than serialization latency of timestamp arriving,
# floor of host clock second will be the navigation epoch sencond.
# Will be used for later join with TIM-TP timestamps.
ticcA["epochSec"] = ticcA["ppsHostClock"].dt.floor("s")
ticcB["epochSec"] = ticcB["ppsHostClock"].dt.floor("s")


In [ ]:
ticcA.dtypes

In [ ]:
ticcA.ppsRefClock.str[2:].astype(float)

In [ ]:
ticcB.dtypes
# XXX next step: string split to int and frac, confirm int part is gap free, convert frac to float